In [1]:
#!/usr/bin/env python3

import pandas as pd
from pathlib import Path
from itertools import count

source = { "Batting":  Path("mnk_batting.csv"), "Pitching": Path("mnk_pitching.csv"), "Fielding": Path("mnk_fielding.csv")}

stat_columns = {
    "Batting": ["G","AB","R","H","2B","3B","HR","RBI","SB","CS","BB","SO","GIDP","PA","TB","AVG","SLG","OBP","OPS","BB_pct","K_pct"],
    "Pitching": ["G","W","L","CG","SHO","SV","IPouts","H","HR","BB","HBP","SO","R","ER","ERA","BFP","WPCT"],
    "Fielding": ["POS","G","PO","A","E","DP","PB","FP","RF"]}

playerStatsID = ["playerID","yearID","lgID","teamID","stint","AL_indicator","CL_indicator","KBO_indicator","NL_indicator","PL_indicator","transition_indicator"]


In [2]:
rows, lookup, uid = [], {}, count(1)

for typ, stats in stat_columns.items():
    for stat in stats:
        stat_id = next(uid)
        rows.append({"statID": stat_id, "statName": stat, "statType": typ})
        lookup[(typ, stat)] = stat_id
        
stats_df = pd.DataFrame(rows)


def pad_id(df):
    for c in playerStatsID:
        if c not in df.columns:
            df[c] = pd.NA
    return df

parts = []
for typ, path in source.items():
    df = pd.read_csv(path)
    df = pad_id(df)
    keep = [s for s in stat_columns[typ] if s in df.columns]
    if not keep:
        continue
    long = (df[playerStatsID + keep].melt(id_vars=playerStatsID, value_vars=keep,var_name="statName", value_name="value").dropna(subset=["value"]))
    long["statType"] = typ
    long["stat"] = long["statName"].apply(lambda s: lookup[(typ, s)])
    parts.append(long[playerStatsID + ["statType","stat","value"]])

C:\Users\Acer\AppData\Local\Temp\ipykernel_26172\3820790886.py:20: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\Acer\AppData\Local\Temp\ipykernel_26172\3820790886.py:20: DtypeWarning: Columns (2,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\Acer\AppData\Local\Temp\ipykernel_26172\3820790886.py:20: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [ ]:
playerstats_df = pd.concat(parts, ignore_index=True)
playerstats_df.insert(0, "playerstatsID", range(1, len(playerstats_df) + 1))
playerstats_df["value"] = (
    pd.to_numeric(playerstats_df["value"].replace("-", pd.NA), errors="coerce"))

stats_df.to_csv("mnk_stats.csv", index=False)
playerstats_df.to_csv("mnk_playerstats.csv", index=False)

ValueError: could not convert string to float: '-'